# Create some generic data with added gausian noise

In [ ]:
# %load imports.py
%load_ext autoreload
%autoreload 2
%config Completer.use_jedi = False  ## (To fix autocomplete)

import pandas as pd
from vessel_manoeuvring_models.models.vmm import ModelSimulator
import matplotlib.pyplot as plt
import matplotlib
#plt.style.use('presentation')

from vessel_manoeuvring_models.visualization.plot import track_plots, plot, captive_plot
import kedro
import numpy as np
import os.path
import anyconfig


from myst_nb import glue
from vessel_manoeuvring_models.symbols import *
import vessel_manoeuvring_models.symbols as symbols
from vessel_manoeuvring_models.system_equations import *

from IPython.display import display, Math, Latex, Markdown
from sympy.physics.vector.printing import vpprint, vlatex

from vessel_manoeuvring_models.models.regression import MotionRegression, Regression

from vessel_manoeuvring_models.parameters import df_parameters
from vessel_manoeuvring_models.substitute_dynamic_symbols import run
from vessel_manoeuvring_models.models.diff_eq_to_matrix import DiffEqToMatrix
p = df_parameters["symbol"]
import statsmodels.api as sm

from vessel_manoeuvring_models.models.force_from_motion import predict_force

# Read configs:
conf_path = os.path.join("../conf/base/")
runs_globals_path = os.path.join(
    conf_path,
    "runs_globals.yml",
)

runs_globals = anyconfig.load(runs_globals_path)
model_test_ids = runs_globals["model_test_ids"]

join_globals_path = os.path.join(
    conf_path,
    "join_globals.yml",
)

joins = runs_globals["joins"]
join_runs_dict = anyconfig.load(join_globals_path)

globals_path = os.path.join(
    conf_path,
    "globals.yml",
)
global_variables = anyconfig.load(globals_path)
from vessel_manoeuvring_models.prime_system import PrimeSystem
from wPCC_pipeline.pipelines.kvlcc2.nodes import calculate_thrust
from wPCC_pipeline.pipelines.motion_regression.nodes import predict_force, fit_motions
from wPCC_pipeline.pipelines.prediction.nodes import simulate_euler
from wPCC_pipeline.pipelines.filter_data_extended_kalman.nodes import extended_kalman_filter, extended_kalman_smoother
from wPCC_pipeline.pipelines.preprocess.nodes import filter

In [ ]:
%reload_kedro

ship='generic_kvlcc2'
id = "MARIN_FREE_KVLCC2_zz_10_m"

raw_data = catalog.load(f"{ship}.{id}.data")

dataframes = {

#"raw_data" : catalog.load(f"{ship}.{id}.data"),   
#"ek" : catalog.load(f"{ship}.initial.{id}.data_ek_filter"),
#"ek2" : catalog.load(f"{ship}.updated.{id}.data_ek_filter"),
"smooth" : catalog.load(f"{ship}.initial.{id}.data_ek_smooth"),
"smooth2" : catalog.load(f"{ship}.updated.{id}.data_ek_smooth"),
"true_data" : catalog.load(f"{ship}.{id}.true_data"),
}

dataframes['lowpass'] = filter(raw_data, cutoff = 1.0)

styles = {

    "true_data":{"style":'k--','zorder':10},
#    "raw_data":{"style":'m-','lw':0.2},
    
    #"ek":{"style":'b.-'},

}

ek1 = catalog.load(f"{ship}.vmm_linear.ek") 

ship_data = catalog.load(f"{ship}.ship_data")


In [ ]:
dataframes_ = {
    'data': raw_data,
    'ek sim': ek1.simulate(data=raw_data, input_columns=['delta','U'])
}

track_plots(dataframes_, lpp=ship_data['L'], beam=ship_data['B'], N=10, plot_boats=True)
plot(dataframes_, fig_size=(15,10), styles=styles, keys=['u','v','r','psi'], time_window=[0,10]);

In [ ]:
track_plots(dataframes, lpp=ship_data['L'], beam=ship_data['B'], N=10, plot_boats=True)

plot(dataframes, fig_size=(15,10), styles=styles, keys=['delta','u','v','r','psi'], time_window=[10,20]);

In [ ]:
len(dataframes['raw_data'])

In [ ]:
len(dataframes['true_data'])

In [ ]:
np.rad2deg(0.001/3)